In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import pandas as pd

class ProcessAmazon():
    """Read, filter, aggregate by ProductId, and generate X matrix"""

    def __init__(self, raw_data_filename, num_reviews_filter):
        """Initialize attributes."""
        self.raw_data_filename = raw_data_filename
        self.num_reviews_filter = num_reviews_filter
        self.raw_data = self.read_and_report()
        self.grouped_by_userid = self.group_by_userid()
        self.filtered = self.filter_()
        self.grouped_by_productid = self.group_by_productid()
        self.X = self.write_X()
    
    def read_and_report(self):
        """Read Amazon csv, report on file structure"""
        data = pd.read_csv(self.raw_data_filename)
        print(self.raw_data_filename, "has", data.shape[0], "rows and", data.shape[1], "columns. First five observations:")
        print(data.head(5))
        print(50 * '-')
        return data
    
    def group_by_userid(self):
        """Group by UserId and look at the distribution of number of reviews"""
        grouped_by_uid = self.raw_data.groupby('UserId').count()
        print('Distribution of number of reviews for each UserId:')
        print(grouped_by_uid['ProductId'].value_counts())
        print(50 * '-')
        return grouped_by_uid
    
    def filter_(self):
        """Filter out reviewers with fewer than x reviews"""
        filtered = self.grouped_by_userid[self.grouped_by_userid['Id'] > self.num_reviews_filter]
        # remove redundant columns
        filtered_final = filtered.drop(filtered.columns[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], axis=1)
        # recreate UserId from index
        filtered_final['UserId'] = filtered_final.index
        # rename columns
        filtered_final.columns = ['num_reviews', 'UserId']
        # left merge filtered list with raw data
        filtered = pd.merge(filtered_final, self.raw_data, how='left', on='UserId')
        print("Filtered data has", filtered.shape[0], "rows and", filtered.shape[1], "columns. First five observations:")
        print(filtered.head(5))
        print(50 * '-')
        return filtered
    
    def group_by_productid(self):
        """Group by ProductId and create a column that lists each UserId (separated by spaces) who reviewed that product"""
        grouped_by_pid = self.filtered.groupby('ProductId')['UserId'].apply(lambda x: ' '.join(x)).reset_index()
        print("Data grouped by ProductId has", grouped_by_pid.shape[0], "rows and", grouped_by_pid.shape[1], "columns. First five observations:")
        print(grouped_by_pid.head(5))
        print(50 * '-')
        return grouped_by_pid
    
    def write_X(self):
        """use bag of words to create boolean X to represent reviews by UserId (one UserId per column)"""
        from sklearn.feature_extraction.text import CountVectorizer
        count = CountVectorizer()
        uidArray = self.grouped_by_productid.UserId.as_matrix()
        userFeatures = count.fit_transform(uidArray)
        print("Shape of final X matrix:", userFeatures.shape)
        return userFeatures

In [18]:
import pandas as pd
class ViewClusteringResults():
    """Merge cluster definitions to raw data and view samples of clustering results, showing review summaries and texts."""
    
    def __init__(self, data_instance, model_instance):
        """Initialize attributes"""
        self.data_instance = data_instance
        self.model_instance = model_instance
    
    def merge_labels(self):
        """Merge cluster labels to raw data (as structured in Amazon.csv)"""
        labels_tomerge = pd.DataFrame(self.model_instance.labels_)
        labelled_products = pd.concat([self.data_instance.grouped_by_productid, labels_tomerge], axis=1, ignore_index=True)
        labelled_products.columns = ['ProductId', 'UserIds', 'cluster']
        merged = pd.merge(labelled_products, self.data_instance.raw_data, how='left', on='ProductId')
        merged_lite = merged[['ProductId', 'cluster', 'Summary', 'Text']]
        return merged_lite

    def view_cluster_samples(self, n_samples):
        """Print x samples of cluster results for each cluster, showing details of product reviews."""
        merged = self.merge_labels()
        num_clusters = self.model_instance.cluster_centers_.shape[0]
        for i in range(1, num_clusters): 
            print(50 * '-')
            print('cluster: ', i)
            cluster = merged.loc[merged['cluster'] == i]
            print('cluster size: ', cluster.shape)
            print(cluster.sample(n_samples))

In [5]:
# process Amazon.csv, filtering out people who left 4 or fewer reviews
amazonData = ProcessAmazon('Amazon.csv', 4)

Amazon.csv has 455000 rows and 13 columns. First five observations:
   Unnamed: 0      Id   ProductId          UserId       ProfileName  \
0      138806  138807  B000E63LME  A1CQGW1AOD0LF2  Alena K. "Alena"   
1      469680  469681  B004ZIH4KM  A37S7U1OX2MCWI        Becky Cole   
2      238202  238203  B003ZXE9QA  A2OM6G73E64EQ9              jeff   
3      485307  485308  B001RVFERK  A25W349EE97NBK          Tangent4   
4      375283  375284  B000OQZNTS  A3CPPW0HUC07YS       Amy Nicolai   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       2      2  1294185600   
1                     0                       0      5  1349740800   
2                     0                       0      5  1329264000   
3                     1                       1      4  1248307200   
4                     0                       0      5  1333238400   

                     Summary  \
0           Not as pictured.   
1                   

In [9]:
# save vectorized features to csv
# np.savetxt("userFeatures.csv", amazonData.X.toarray(), delimiter=",") -- too slow

KeyboardInterrupt: 

In [10]:
# first attempt at fitting K means to view change in Inertia
from sklearn.cluster import KMeans

# container to store inertia scores over iterations
distortions = []

# fit KMeans iteratively to begin to assess the appropriate number of clusters
for i in range(41, 60):
    km = KMeans(n_clusters=i)
    km.fit(amazonData.X)
    distortions.append(km.inertia_)

KeyboardInterrupt: 

In [ ]:
# vizualize change in inertia
plt.plot(range(41, 60), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [11]:
# fit KMeans with 10 clusters
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3000)
km.fit(amazonData.X)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3000,
    n_init=10, n_jobs=1, precompute_distances='auto', random_state=None,
    tol=0.0001, verbose=0)

In [12]:
# create an instance of ViewClusteringResults to view results of final KMeans fit
viz = ViewClusteringResults(amazonData, km)

In [13]:
# view samples of product reviews, organized by KMeans cluster results
viz.view_cluster_samples(10)

--------------------------------------------------
cluster:  1
cluster size:  (137, 4)
       ProductId  cluster                                Summary  \
9308  B00020HHHC        1                        A very fine Tea   
9292  B00020HHHC        1   People Either Love or Hate Valerian!   
9294  B00020HHHC        1            Great herbal tea for cramps   
9337  B00020HHHC        1     Recommended by herbalist--love it.   
9343  B00020HHHC        1                          great product   
9276  B00020HHHC        1   Bad taste, but helps you fall asleep   
9390  B00020HHHC        1                 Fragrant chamomile tea   
9345  B00020HHHC        1                                    NaN   
9302  B00020HHHC        1                       Menopause Relief   
9380  B00020HHHC        1  very pleasant flavor and good quality   

                                                   Text  
9308  This tea was very good. I was Very happy I got...  
9292  I love Valerian and I love it's earthy sme

ValueError: Cannot take a larger sample than population when 'replace=False'

In [14]:
# fit KMeans with 1000 clusters - 3000 was overfit (clustered by product not by type of product)
from sklearn.cluster import KMeans
km1000 = KMeans(n_clusters=1000)
km1000.fit(amazonData.X)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=1000,
    n_init=10, n_jobs=1, precompute_distances='auto', random_state=None,
    tol=0.0001, verbose=0)

In [19]:
# create an instance of ViewClusteringResults to view results of final KMeans fit
viz = ViewClusteringResults(amazonData, km1000)
# view samples of product reviews, organized by KMeans cluster results
viz.view_cluster_samples(50)

--------------------------------------------------
cluster:  1
cluster size:  (258, 4)
         ProductId  cluster  \
119792  B000WFN0VO        1   
119350  B000WFKI82        1   
119337  B000WFKI82        1   
119697  B000WFN0VO        1   
119721  B000WFN0VO        1   
119360  B000WFKI82        1   
119668  B000WFN0VO        1   
119735  B000WFN0VO        1   
119367  B000WFKI82        1   
119664  B000WFN0VO        1   
119393  B000WFKI82        1   
119412  B000WFKI82        1   
119397  B000WFKI82        1   
119352  B000WFKI82        1   
119675  B000WFN0VO        1   
119414  B000WFKI82        1   
119683  B000WFN0VO        1   
119326  B000WFKI82        1   
119674  B000WFN0VO        1   
119371  B000WFKI82        1   
119387  B000WFKI82        1   
119745  B000WFN0VO        1   
119783  B000WFN0VO        1   
119298  B000WFKI82        1   
119400  B000WFKI82        1   
119700  B000WFN0VO        1   
119691  B000WFN0VO        1   
119308  B000WFKI82        1   
119699  B000WF

ValueError: Cannot take a larger sample than population when 'replace=False'